<a href="https://colab.research.google.com/github/wz30/duke_edge_computing/blob/main/Wei_Federated_Learning_on_MNIST_using_a_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Federated Learning on MNIST using a CNN with PyTorch & PySyft


### Context 

Federated learning is a machine learning technique that trains an algorithm across multiple decentralized edge devices or servers holding local data samples, without exchanging their data samples. 


We use Google Colaboratory to execute code
###related reference: 
PyTorch(https://github.com/pytorch/examples/blob/master/mnist/main.py)

PySyft (https://github.com/OpenMined/PySyft/)

Colaboratory (https://colab.research.google.com/) 


Colaboratory support for importing a library that's not in Colaboratory by default. In this tutorial,we just need install syft package by pip.

In [ ]:
! pip install syft==0.2.9

     |████████████████████████████████| 440kB 5.6MB/s 
     |████████████████████████████████| 20.1MB 47.5MB/s 
     |████████████████████████████████| 491kB 39.2MB/s 
     |████████████████████████████████| 450kB 35.7MB/s 
     |████████████████████████████████| 133kB 49.8MB/s 
     |████████████████████████████████| 81kB 6.5MB/s 
     |████████████████████████████████| 1.8MB 43.7MB/s 
     |████████████████████████████████| 2.0MB 35.5MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 2.2MB 34.9MB/s 
     |████████████████████████████████| 9.0MB 49.4MB/s 
     |████████████████████████████████| 753.4MB 21kB/s 
     |████████████████████████████████| 61kB 5.6MB/s 
     |████████████████████████████████| 4.0MB 30.1MB/s 
     |████████████████████████████████| 204kB 31.1MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 37.2MB 84kB/s 
   

In [ ]:
# !pip install torch==1.4.0

### Imports and model specifications

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import sys
#torch version should be 1.4.0 to be compatiable with syft
print(torch.__version__)
print(sys.version)

1.4.0
3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]


And than those specific to PySyft. In particular we define remote workers `alice` and `bob`.

In [ ]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

We define the setting of the learning task

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 2
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

### Data loading and sending to workers
We first load the data and transform the training Dataset into a Federated Dataset split across the workers using the `.federate` method. This federated dataset is now given to a Federated DataLoader. The test dataset remains unchanged.

In [ ]:
mnist_train = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

In [ ]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [ ]:
print(len((test_loader.dataset)[0][0][0]))


28


In [ ]:

t1 = torch.tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])
t2 = torch.tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])
t1.size()
t3 = torch.cat(
    (t1,t2)
    ,dim=0
)
t4 = torch.cat(
    (t3,t2)
    ,dim=0
)
t4 


tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])

In [ ]:
from tree import tree_benchmark
print(tree_benchmark.TIME_UNITS)
print(type("str"))

[(1, 's'), (0.001, 'ms'), (1e-06, 'us'), (1e-09, 'ns')]
<class 'str'>


In [ ]:
tup = [(1,2),(2,3),(4,1),(1,2),(2,3),(4,1),(1,2),(2,3),(4,1),(7,5)]

[x for (x,y) in tup][int(len(tup)*0.3):]

[1, 2, 4, 1, 2, 4, 7]

In [ ]:
torch.utils.data.DataLoader?

### CNN specification
Here we use exactly the same CNN as in the official example.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Define the train and test functions
For the train function, because the data batches are distributed across `alice` and `bob`, you need to send the model to the right location for each batch. Then, you perform all the operations remotely with the same syntax like you're doing local PyTorch. When you're done, you get back the model updated and the loss to look for improvement.

In [ ]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

The test function does not change!

In [ ]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### Launch the training !

In [ ]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")



Train Epoch: 1 [0/60032 (0%)]	Loss: 2.303102
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.210071
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.968947
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.459771
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.979385
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.749077
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.573528
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.583597
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.422385
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.480115
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.375935
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.345905
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.416229
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.342528
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.285184
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.230878
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.205240
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.415740
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.305404
Train Epoch: 1 [36480/60032 (61%)]	Loss: 0.147625
Train Epoch: